In [5]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [6]:

# creating the class
class TitanicMl(pd.DataFrame):
    def normalize_cols(self):
        '''
        cleaning columns
        '''
        self.columns = self.columns.str.replace(" ", "_")
        self.columns = self.columns.str.lower()
        return(self)

    
    def clean_no_values(self, col_name):
        '''getting rid of (No Value) '''
        amount_cleaning = [" (No Value) ", r"\(.*\)", '\s+$']  
        for item in amount_cleaning:
            self[col_name] = self[col_name].str.replace(item, "")
        return(self)
    
    def deal_owner_cleaning(self):
        '''requires clean columns'''
        try:
            deal_owner_cleaning = [" (Deactivated User) ", r"\(.*\)", '\s+$']  
            for item in deal_owner_cleaning:
                self["deal_owner"] = self["deal_owner"].str.replace(item, "")
        except:
            print("KEY ERROR: Please make sure 'deal_owner' is in the df")


In [7]:
# Reading in Data
test_df = pd.read_csv(os.path.join("resources","test.csv"))
train_df = pd.read_csv(os.path.join("resources","train.csv"))

parch = # of parents / children aboard the Titanic
<br>
sibsp = # of siblings / spouses aboard the Titanic
<br>
survival = {0 = No, 1 = Yes}

In [10]:
train_df = TitanicMl(train_df)
train_df = train_df.normalize_cols()
train_df

,passengerid,survived,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [9]:
train_df.describe()

train_df.

SyntaxError: invalid syntax (3179322204.py, line 3)

Age